<a href="https://colab.research.google.com/github/hosamfa3096/Class_AI3_hos/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalación de herramientas

In [ ]:
!pip install laserembeddings

!python -m laserembeddings download-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 859 kB 47.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=3145c8c88ef5a2be3d1e66e62e09d86320b6e058e12d5285627f975d63880245
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
!pip install bpmll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk import download as nltkdownload
nltkdownload('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Carga de los datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sem_eval_train_es.csv', quotechar='"', skipinitialspace=True)

In [ ]:
ds_tw=pd.DataFrame(df['Tweet']).astype(str) #Creamos un dataset con los tweets únicamente.

In [ ]:
ds_tw.head(20)

,Tweet
0,@aliciaenp Ajajjaa somos del clan twitteras pe...
1,@AwadaNai la mala suerte del gato fichame la c...
2,@audiomano A mí tampoco me agrado mucho eso. E...
3,Para llevar a los bebes de un lugar a otro deb...
4,@DalasReview me encanta la terrible hipocresia...
5,"En Venezuela, el tráfico aéreo esta tan peligr..."
6,me voy a morir sin saber por qué @. y @.. y @....
7,@tctelevision @calle7tc Terrible esto!
8,"Para tristeza y felicidad, Ed Sheeran al resca..."
9,Con razon mi vieja se fue rapido a Santa Fe si...


In [ ]:
ds_sem=pd.DataFrame(df.iloc[:, 2:13]).astype(int) #Creamos un dataset con los sentimientos a predecir, nuestro objetivo.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(ds_tw, ds_sem, test_size=0.20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2848, 1) (713, 1) (2848, 11) (713, 11)


In [ ]:
import numpy as np



X_train = np.array(X_train)
Y_train = np.array(y_train) 

print(X_train[0])
print(Y_train[0])
print(Y_train.shape, "\n")

X_test = np.array(X_test)
Y_test = np.array(y_test)

print(X_test[0])
print(Y_test[0])
print(Y_test.shape, "\n")

['No puedo de la tristeza, que mierda todo.']
[0 0 0 0 0 0 0 0 1 0 0]
(2848, 11) 

['@mpz1005 Jajajaja es una comedia española, chistosa pero salen españolas muy guapas la verdad. Jajajaa']
[0 0 0 0 1 0 0 0 0 0 0]
(713, 11) 



In [ ]:
count = 0
for element in Y_train:
  if np.sum(element) == 0:
    count +=1
print(count)


127


In [ ]:
Y_train_clean = np.array([item for item in Y_train if np.sum(item)>0])
print("Cleaned Y_train from ", len(Y_train), " to ", len(Y_train_clean))

X_train_clean = np.array([X_train[i] for i, item in enumerate(Y_train) if np.sum(item)>0])
print("Cleaned X_train from ", len(X_train), " to ", len(X_train_clean))

Y_test_clean = np.array([item for item in Y_test if np.sum(item)>0])
print("Cleaned Y_test from ", len(Y_test), " to ", len(Y_test_clean))

X_test_clean = np.array([X_test[i] for i, item in enumerate(Y_test) if np.sum(item)>0])
print("Cleaned X_test from ", len(X_test), " to ", len(X_test_clean))

Cleaned Y_train from  2848  to  2721
Cleaned X_train from  2848  to  2721
Cleaned Y_test from  713  to  676
Cleaned X_test from  713  to  676


In [ ]:
accum = sum(Y_train_clean)
print(np.around(accum/len(Y_train_clean),decimals=2))

[0.34 0.12 0.15 0.12 0.31 0.08 0.1  0.17 0.25 0.05 0.05]


##Generamos los 'embeddings' de las frases

In [ ]:
from laserembeddings import Laser

laser = Laser()

X_train_embeddings = laser.embed_sentences(X_train,lang='es')
X_test_embeddings = laser.embed_sentences(X_test,lang='es')

In [ ]:
print(X_train_embeddings.shape)
print(X_test_embeddings.shape)

(2848, 1024)
(713, 1024)


##Creación de los clasificadores

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

n = X_train_embeddings.shape[0]
dim_no = X_train_embeddings.shape[1]
class_no = Y_train.shape[1]



def create_baseline(number_of_classes=class_no):
  model = Sequential()
  model.add(Dense(128, input_dim=dim_no, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(number_of_classes, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']


stats = []
for i in range(11):
  ordinal = i
  X_input = X_train_embeddings
  Y_output = Y_train[:,ordinal]

  estimator = KerasClassifier(build_fn=create_baseline, number_of_classes=1, epochs=5, batch_size=128, verbose=0)
  kfold = StratifiedKFold(n_splits=4, shuffle=True)
  results = cross_val_score(estimator, X_input, Y_output, cv=kfold)
  print(labels[i], "baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
  stats.append(results.mean())

print("average: %.2f%% (%.2f%%)" % (np.mean(stats)*100, np.std(stats)*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


anger baseline: 67.66% (0.83%)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


anticipation baseline: 88.34% (0.00%)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


...

sadness baseline: 76.23% (0.06%)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


surprise baseline: 94.91% (0.06%)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


trust baseline: 95.26% (0.06%)
average: 84.85% (9.09%)


In [ ]:
#Probamos el modelo en una dimensión
selection = 8
model = create_baseline(number_of_classes=1)
model.fit(X_train_embeddings, Y_train[:,selection], epochs=20, validation_data=(X_test_embeddings, Y_test[:,selection]))

Epoch 1/20
89/89 [==============================] - 1s 5ms/step - loss: 0.5517 - accuracy: 0.7588 - val_loss: 0.5231 - val_accuracy: 0.7644
Epoch 2/20
89/89 [==============================] - 0s 3ms/step - loss: 0.5205 - accuracy: 0.7623 - val_loss: 0.5078 - val_accuracy: 0.7644
Epoch 3/20
89/89 [==============================] - 0s 3ms/step - loss: 0.5042 - accuracy: 0.7623 - val_loss: 0.4924 - val_accuracy: 0.7644
Epoch 4/20
89/89 [==============================] - 0s 3ms/step - loss: 0.4819 - accuracy: 0.7651 - val_loss: 0.4748 - val_accuracy: 0.7686
Epoch 5/20
89/89 [==============================] - 0s 3ms/step - loss: 0.4582 - accuracy: 0.7763 - val_loss: 0.4601 - val_accuracy: 0.7784
Epoch 6/20
89/89 [==============================] - 0s 3ms/step - loss: 0.4346 - accuracy: 0.7995 - val_loss: 0.4491 - val_accuracy: 0.7882
Epoch 7/20
89/89 [==============================] - 0s 3ms/step - loss: 0.4135 - accuracy: 0.8157 - val_loss: 0.4382 - val_accuracy: 0.8121
Epoch 8/20
89/89 [==

In [ ]:
models = []
for i in range(class_no):
  model = create_baseline(number_of_classes=1)
  Y_train_split = Y_train[:,i]
  print("processing model for",labels[i])
  model.fit(X_train_embeddings, Y_train_split, epochs=10, verbose=0)
  models.append(model)

print("trained", len(models), "models")

processing model for anger
processing model for anticipation
processing model for disgust
processing model for fear
processing model for joy
processing model for love
processing model for optimism
processing model for pessimism
processing model for sadness
processing model for surprise
processing model for trust
trained 11 models


##Evaluación del modelo


In [ ]:
print("Evaluate on test data")
results=[]
for i, model in enumerate(models):
  Y_test_split = Y_test[:,i]
  partial = model.evaluate(X_test_embeddings, Y_test_split, batch_size=128, verbose=0)
  print(labels[i], "test loss, test acc:", partial)


Evaluate on test data
anger test loss, test acc: [0.5064172148704529, 0.739130437374115]
anticipation test loss, test acc: [0.3081320822238922, 0.8835904598236084]
disgust test loss, test acc: [0.36215218901634216, 0.8541374206542969]
fear test loss, test acc: [0.19361209869384766, 0.9270687103271484]
joy test loss, test acc: [0.45271727442741394, 0.799439013004303]
love test loss, test acc: [0.18222084641456604, 0.9326788187026978]
optimism test loss, test acc: [0.3351530432701111, 0.8681626915931702]
pessimism test loss, test acc: [0.3791292607784271, 0.844319760799408]
sadness test loss, test acc: [0.4278205335140228, 0.8218793869018555]
surprise test loss, test acc: [0.1340196579694748, 0.9663394093513489]
trust test loss, test acc: [0.20453302562236786, 0.9438990354537964]


In [ ]:
# Calculamos el F1 score para una dimensión

true = Y_test[:,selection]
prediction = models[selection].predict(X_test_embeddings)

muestra = 10


from sklearn.preprocessing import Binarizer
transformer = Binarizer(threshold=0.5).fit(prediction)  
prediction = transformer.transform(prediction)
flat = [x[0] for x in prediction]


from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
recall = recall_score(y_true=true, y_pred=flat, average='weighted')
precision = precision_score(y_true=true, y_pred=flat, average='weighted')
score = f1_score(y_true=true, y_pred=flat, average='weighted')

print(recall, precision, score, "\n")

for i in range(muestra):
  print(X_test[i])
  print("predicted",labels[selection], bool(flat[i]))
  print("expected",labels[selection], bool(true[i]), "\n")

0.8218793828892006 0.8157849154162424 0.7939124782154668 

['@mpz1005 Jajajaja es una comedia española, chistosa pero salen españolas muy guapas la verdad. Jajajaa']
predicted sadness False
expected sadness False 

['@albertoperezs90 Ya aguantas más que yo y eso que no fumo ']
predicted sadness False
expected sadness False 

['He visto dos reacciones de hombres al comprar un anillo de compromiso: El emocionado y el indeciso  Trata de ser el primero']
predicted sadness False
expected sadness False 

['@TamezGaby Cada día te ves más hermosa y que hermosa sonrisa tienes saludos desde aquí cerca desde Cozumel bendiciones 😘']
predicted sadness False
expected sadness False 

['tuve un lindo domingo ']
predicted sadness False
expected sadness False 

['Dudes! Neta que se necesita estar bien pendejitos de la cabeza si piensan que con snaps asquerosos provocarán algo en una mujer.']
predicted sadness False
expected sadness False 

['Les juro que mi desesperación es demasiada, a cada oxxo o 7-el

In [ ]:
#Calculamos el F1 score

for i in range(class_no):
  true = Y_test[:,i]
  prediction = models[i].predict(X_test_embeddings)

  from sklearn.preprocessing import Binarizer
  transformer = Binarizer(threshold=0.5).fit(prediction)  
  prediction = transformer.transform(prediction)

  from sklearn.metrics import f1_score, recall_score, precision_score

  recall = recall_score(y_true=true, y_pred=prediction, average='weighted')
  precision = precision_score(y_true=true, y_pred=prediction, average='weighted', zero_division=0)
  score = f1_score(y_true=true, y_pred=prediction, average='weighted')

  print("%.2f %.2f %.2f" % (recall, precision, score), labels[i])

0.74 0.74 0.74 anger
0.88 0.78 0.83 anticipation
0.85 0.83 0.80 disgust
0.93 0.90 0.90 fear
0.80 0.81 0.78 joy
0.93 0.92 0.91 love
0.87 0.75 0.81 optimism
0.84 0.71 0.77 pessimism
0.82 0.82 0.79 sadness
0.97 0.93 0.95 surprise
0.94 0.89 0.92 trust


##Testeo en el conjunto ciego de test

In [ ]:
blank_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sem_eval_test_blank_es.csv', quotechar='"', skipinitialspace=True)

In [ ]:
final_test= np.array(blank_df['Tweet'])

In [ ]:
final_test_emb = laser.embed_sentences(final_test,lang='es')

In [ ]:
from sklearn.preprocessing import Binarizer
transformer = Binarizer(threshold=0.5).fit(prediction)  
prediction = transformer.transform(prediction)
flat = [x[0] for x in prediction]


for i, sentence in enumerate(final_test):
  print("\n", sentence)
  single_embedding = np.array([final_test_emb[i]])
  for j, model in enumerate(models):
    prediction = model.predict(single_embedding)  
    normalized = transformer.transform(prediction)
    result = bool(normalized[0,0])
    if result: print("guessed", labels[j], "with prob", prediction[0,0])


 No me pienso perder la pelea de McGregor contra Myweather
guessed anger with prob 0.6765298

 Yo preocupada pensando que mi papá ya había llegado y me estaba llamando para decirme que el vuelo todavía no había salido 👍
guessed sadness with prob 0.6720923

 Pucha ya no me sirven todos los carros a Rondizzoni.... #ironía jajajjajaja #unañomas
guessed joy with prob 0.8271379

 Si estar contigo es un delito, hago cien años en prision 😃
guessed anger with prob 0.6325409

 @macacifuentesC @sergmujica Perfect... Y de una mansa ni que terrible PLR

 mera dale estoy aburrido
guessed sadness with prob 0.6731826

 El sol , los perros y una birra ! #felicidad
guessed joy with prob 0.78564703

 Tantos cambios y la vida no te pregunta si los quieres, casi siempre los impone y las dos opciones que quedan es aceptar o seguir llorando.
guessed sadness with prob 0.66199934

 En estos momentos siento mi cuerpo hirviendo.
guessed anger with prob 0.5231783
guessed sadness with prob 0.64928764

 Buenas no

In [ ]:
resultados_finales=[]
transformer = Binarizer(threshold=0.6)

for j, model in enumerate(models):
  predicciones=[]
  for i, sentence in enumerate(final_test):
    single_embedding = np.array([final_test_emb[i]])
    prediction = model.predict(single_embedding)
    normalized= transformer.transform(prediction)
    result = bool(normalized[0,0])
    predicciones.append(result)
  resultados_finales.append(predicciones) 
    
print(resultados_finales)


[[True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, True, True, True, True, True, False, False, True, False, False, False, False, True, True, True, False, False, True, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, True, False, False, True, False, False, True, False, False, True, True, False, False, False, False, True, True, False, False, False, True, True, True, True, False, True, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, True, True, True, True, True, False, False, False, True, False, True, True, True, False, False, True, False, True, False, False, True, False, False, True, True, False, False, False, False, True, False

In [ ]:
print(labels)

['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']


In [ ]:
blank_df['anger']= resultados_finales[0]
blank_df['anticipation']= resultados_finales[1]
blank_df['disgust']= resultados_finales[2]
blank_df['fear']= resultados_finales[3]
blank_df['joy']= resultados_finales[4]
blank_df['optimism']= resultados_finales[5]
blank_df['sadness']= resultados_finales[6]
blank_df['surprise']= resultados_finales[7]
blank_df['trust']= resultados_finales[8]


In [ ]:
blank_df.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'optimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [ ]:
blank_df

,ID,Tweet,anger,anticipation,disgust,fear,joy,optimism,sadness,surprise,trust
0,2018-Es-06697,No me pienso perder la pelea de McGregor contr...,True,False,False,False,False,False,False,False,False
1,2018-Es-05385,Yo preocupada pensando que mi papá ya había ll...,False,False,False,False,False,False,False,False,True
2,2018-Es-03777,Pucha ya no me sirven todos los carros a Rondi...,False,False,False,False,True,False,False,False,False
3,2018-Es-02175,"Si estar contigo es un delito, hago cien años ...",True,False,False,False,False,False,False,False,False
4,2018-Es-00726,@macacifuentesC @sergmujica Perfect... Y de un...,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
674,2018-Es-06551,@tccharless -Peligro al volante -Tec -Los San...,False,False,False,False,False,False,False,False,False
675,2018-Es-02782,Vamos a ver q sucedio en mi barrio en los 90 #...,False,False,False,False,False,False,False,False,False
676,2018-Es-01308,@kopexinfinity FELICIDADES FER 💕 QUE ESTA PASA...,False,False,False,False,True,False,False,False,False
677,2018-Es-02136,@PabloEmanuel299 LAMENTO MI CONFUSIÓN DE RAMA ...,True,False,False,False,False,False,False,False,False


In [ ]:
from google.colab import drive
drive.mount('drive')
blank_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Resultados_NLP.csv')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
